# Automatic Community Detection in Bipartite Networks

This notebook demonstrates the `CommunityDetector` class, which applies the Sanguinetti, Laidler & Lawrence (2005) automatic spectral clustering algorithm to bipartite networks (e.g., country-product networks).

## Key Features

- Uses ECI/PCI-style **transition matrix affinity**: $T = D_c^{-1} M D_p^{-1} M^T$
- **Automatic cluster detection** via origin detector (same algorithm as `SpectralCluster`)
- **Elongated k-means** for eigenvector clustering
- **Iterative eigenvector expansion** until optimal number found

## Comparison to Point Cloud Clustering

- `SpectralCluster`: Point cloud data → Gaussian affinity → eigenvectors → clusters
- `CommunityDetector`: Bipartite network → Transition matrix → eigenvectors → communities

Both use the **same core algorithm** for automatic detection!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import sparse
import sys
sys.path.insert(0, '..')

from fitkit.community import CommunityDetector

%matplotlib inline
plt.rcParams['figure.figsize'] = (15, 5)

## Generate Synthetic Bipartite Network with Communities

We'll create a country-product network with 3 distinct communities:
- Each community has countries that specialize in certain products
- Strong within-community connections
- Weak between-community connections

In [ ]:
def generate_community_network(n_communities=3, countries_per_comm=20, 
                                products_per_comm=15, within_prob=0.7, 
                                between_prob=0.05, seed=42):
    """
    Generate a bipartite network with clear community structure.
    
    Parameters
    ----------
    n_communities : int
        Number of communities
    countries_per_comm : int
        Countries per community
    products_per_comm : int
        Products per community
    within_prob : float
        Connection probability within community
    between_prob : float
        Connection probability between communities
    
    Returns
    -------
    M : ndarray
        Bipartite adjacency matrix
    true_labels : ndarray
        Ground truth community labels for countries
    """
    np.random.seed(seed)
    
    n_countries = n_communities * countries_per_comm
    n_products = n_communities * products_per_comm
    
    M = np.zeros((n_countries, n_products))
    true_labels = np.repeat(np.arange(n_communities), countries_per_comm)
    
    # Assign products to communities
    product_communities = np.repeat(np.arange(n_communities), products_per_comm)
    
    for c in range(n_countries):
        c_comm = true_labels[c]
        
        for p in range(n_products):
            p_comm = product_communities[p]
            
            if c_comm == p_comm:
                # Within community: high probability
                if np.random.random() < within_prob:
                    M[c, p] = 1
            else:
                # Between communities: low probability
                if np.random.random() < between_prob:
                    M[c, p] = 1
    
    return M, true_labels


# Generate network
M, true_labels = generate_community_network(
    n_communities=3,
    countries_per_comm=20,
    products_per_comm=15,
    within_prob=0.7,
    between_prob=0.05,
    seed=42
)

print(f"Network shape: {M.shape}")
print(f"Network density: {M.sum() / M.size:.2%}")
print(f"True communities: {len(np.unique(true_labels))}")
print(f"Countries per community: {np.bincount(true_labels)}")

In [ ]:
# Visualize the network structure
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Raw adjacency matrix
axes[0].imshow(M, cmap='Blues', aspect='auto', interpolation='nearest')
axes[0].set_xlabel('Products')
axes[0].set_ylabel('Countries')
axes[0].set_title('Bipartite Adjacency Matrix M', fontweight='bold')
axes[0].grid(False)

# Sorted by true communities (for visualization)
sorted_idx = np.argsort(true_labels)
M_sorted = M[sorted_idx, :]
axes[1].imshow(M_sorted, cmap='Blues', aspect='auto', interpolation='nearest')
axes[1].set_xlabel('Products')
axes[1].set_ylabel('Countries (sorted by community)')
axes[1].set_title('Matrix Sorted by True Communities', fontweight='bold')
axes[1].grid(False)

# Add community boundaries
for comm_id in range(1, 3):
    boundary = comm_id * 20 - 0.5
    axes[1].axhline(boundary, color='red', linewidth=2, linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

## Apply Automatic Community Detection

Use `CommunityDetector` to automatically find the number of communities.

In [ ]:
# Apply community detection
print("Running CommunityDetector...\n")
detector = CommunityDetector(
    lambda_elongation=0.2,  # Elongation parameter
    max_communities=10,     # Maximum to consider
    random_state=42,
    affinity='bipartite'    # Use bipartite transition matrix
)
detector.fit(M)

print(f"\n{'='*60}")
print(f"RESULT: Community Detection")
print(f"{'='*60}")
print(f"Detected communities: {detector.n_communities_}")
print(f"Expected communities: {len(np.unique(true_labels))}")
print(f"Status: {'✓ PASS' if detector.n_communities_ == 3 else '? REVIEW'}")
print(f"\nCommunity sizes: {np.bincount(detector.labels_)}")
print(f"Eigenvectors used: {detector.eigenvectors_.shape[1]}")
print(f"{'='*60}\n")

## Visualize Results

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Ground truth
sorted_true = np.argsort(true_labels)
M_sorted_true = M[sorted_true, :]
axes[0].imshow(M_sorted_true, cmap='Blues', aspect='auto', interpolation='nearest')
axes[0].set_xlabel('Products')
axes[0].set_ylabel('Countries')
axes[0].set_title('Ground Truth (3 communities)', fontweight='bold')
for comm_id in range(1, 3):
    axes[0].axhline(comm_id * 20 - 0.5, color='red', linewidth=2, linestyle='--', alpha=0.7)

# Predicted
sorted_pred = np.argsort(detector.labels_)
M_sorted_pred = M[sorted_pred, :]
axes[1].imshow(M_sorted_pred, cmap='Blues', aspect='auto', interpolation='nearest')
axes[1].set_xlabel('Products')
axes[1].set_ylabel('Countries')
axes[1].set_title(f'Predicted ({detector.n_communities_} communities)', 
                  fontweight='bold',
                  color='green' if detector.n_communities_ == 3 else 'orange')
# Add predicted boundaries
boundaries = np.where(np.diff(detector.labels_[sorted_pred]) != 0)[0] + 0.5
for boundary in boundaries:
    axes[1].axhline(boundary, color='blue', linewidth=2, linestyle='--', alpha=0.7)

# Eigenspace (first 2 eigenvectors)
axes[2].scatter(detector.eigenvectors_[:, 0], detector.eigenvectors_[:, 1],
                c=detector.labels_, cmap='tab10', s=50, alpha=0.6)
axes[2].scatter(detector.centers_[:, 0], detector.centers_[:, 1],
                c='black', marker='D', s=200, edgecolors='white', linewidths=2,
                label='Cluster Centers', zorder=5)
axes[2].set_xlabel('Eigenvector 1')
axes[2].set_ylabel('Eigenvector 2')
axes[2].set_title('Eigenspace Projection', fontweight='bold')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Community Purity Analysis

Measure how well predicted communities match ground truth.

In [ ]:
from collections import Counter

print("Community Purity Analysis:")
print("="*60)
total_purity = 0
for pred_comm in range(detector.n_communities_):
    mask = detector.labels_ == pred_comm
    true_in_comm = true_labels[mask]
    most_common = Counter(true_in_comm).most_common(1)[0]
    purity = most_common[1] / len(true_in_comm) * 100
    total_purity += most_common[1]
    print(f"Predicted community {pred_comm}: {len(true_in_comm):3d} countries, "
          f"{purity:5.1f}% from true community {most_common[0]}")

overall_purity = total_purity / len(true_labels) * 100
print("="*60)
print(f"Overall purity: {overall_purity:.1f}%")
print("="*60)

## Connection to ECI/PCI

The transition matrix used by `CommunityDetector` is the **same matrix** that ECI/PCI analysis uses:

$$T = D_c^{-1} M D_p^{-1} M^T$$

- **ECI**: Uses the 2nd eigenvector of $T$ as a country ranking
- **CommunityDetector**: Uses multiple eigenvectors of $T$ to find communities

Let's verify this by computing ECI and comparing to the detected communities:

In [ ]:
from fitkit.algorithms import ECI

# Compute ECI
eci_estimator = ECI()
eci, pci = eci_estimator.fit_transform(sparse.csr_matrix(M))

# Plot ECI values colored by detected community
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ECI by detected community
for comm_id in range(detector.n_communities_):
    mask = detector.labels_ == comm_id
    axes[0].scatter(np.where(mask)[0], eci[mask], 
                   label=f'Community {comm_id}', s=50, alpha=0.7)
axes[0].set_xlabel('Country Index')
axes[0].set_ylabel('ECI')
axes[0].set_title('ECI Values by Detected Community', fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# ECI vs first eigenvector from community detection
# Note: ECI uses 2nd eigenvector, but should correlate with eigenspace structure
axes[1].scatter(eci, detector.eigenvectors_[:, 0], 
               c=detector.labels_, cmap='tab10', s=50, alpha=0.6)
axes[1].set_xlabel('ECI (2nd eigenvector of T)')
axes[1].set_ylabel('1st eigenvector (community detection)')
axes[1].set_title('ECI vs Community Detection Eigenspace', fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nNote: ECI uses the 2nd eigenvector for ranking, while")
print("community detection uses multiple eigenvectors for clustering.")
print("Both work with the same underlying transition matrix T!")

## Summary

This notebook demonstrates:

1. ✓ **Automatic community detection** in bipartite networks using `CommunityDetector`
2. ✓ Uses the **same algorithm** as `SpectralCluster` (origin detector + elongated k-means)
3. ✓ Works with **ECI/PCI-style transition matrices** (bipartite affinity)
4. ✓ Correctly identifies 3 communities with high purity
5. ✓ Connects to ECI/PCI analysis (same underlying matrix $T$)

The key insight: **`SpectralCluster` and `CommunityDetector` use the same core algorithm**, just with different affinity matrices:
- `SpectralCluster`: Gaussian affinity on point clouds
- `CommunityDetector`: Transition matrix on bipartite networks